In [1]:
"""
Peatland Protection  Map
Shows protected/unprotected peatlands in Northern Ireland with clickable popups.
Farms and trails are visible but not interactive.
"""
import geopandas as gpd                    # for spatial data handling
import pandas as pd                        # for dataframes and joining
import folium                              # for interactive mapping
from shapely.geometry import LineString    # for distance line drawing


In [2]:
def load_and_prepare_data():
    """
    Load and reproject all spatial datasets to EPSG:4326.
    """
    peat = gpd.read_file("datafiles/Peatland/Priority_Habitats_-_Peatland.shp").to_crs(epsg=4326)    # peatland polygons
    assi = gpd.read_file("datafiles/ASSI/ASSI.shp").to_crs(epsg=4326)                                # protected ASSI areas
    aonb = gpd.read_file("datafiles/AONB/AONB.shp").to_crs(epsg=4326)                                # AONB areas
    trails = gpd.read_file("datafiles/Trails/offroadtrailsni.shp").to_crs(epsg=4326)                 # trails network
    scheme_data = gpd.read_file("datafiles/Farm/SchemeData.shp").to_crs(epsg=4326)                   # farm scheme data

    return peat, assi, aonb, trails, scheme_data
peat, assi, aonb, trails, scheme_data = load_and_prepare_data()

In [3]:
"""
  Confirm peat is in EPSG:4326 for web display
    """
peat.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
"""
  Confirm assi is in EPSG:4326 for web display
    """
assi.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
"""
  Confirm aonb is in EPSG:4326 for web display
    """
aonb.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
"""
  Confirm trails is in EPSG:4326 for web display
    """
trails.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
"""
  Confirm scheme_data is in EPSG:4326 for web display
    """
scheme_data.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
"""
Preview the first 10 rows of the peat GeoDataFrame to inspect structure and key attributes
    """
peat.head(10)

,Hab_Type,Area_Hecta,Area_Sq_Km,GlobalID,OBJECTID,Shape__Are,Shape__Len,geometry
0,Peatland,8.8,0.1,d2ee3bdf-4909-4fcf-8b57-c70bb72cdffb,1,87866.112747,1824.382902,"POLYGON ((-6.28296 55.29384, -6.28299 55.29394..."
1,Peatland,3.9,0.0,02f13c90-725b-4220-bd43-a50d32dad072,2,39352.516373,1003.656513,"POLYGON ((-6.27415 55.29691, -6.27396 55.29785..."
2,Peatland,5.7,0.1,500ab4aa-e3c9-4e1d-b684-eaaf4bb7fc66,3,56876.023880,1567.593824,"POLYGON ((-6.26938 55.29581, -6.26938 55.29588..."
3,Peatland,15.2,0.2,6fc03354-77c0-4ae3-814b-e78ccf97efb2,4,151575.238464,2904.839604,"POLYGON ((-6.18989 55.25893, -6.18985 55.25889..."
4,Peatland,28.3,0.3,5e901074-7ed7-4711-92db-063c02d8b1df,5,283366.382401,5176.676286,"POLYGON ((-6.19425 55.2632, -6.19447 55.26344,..."
5,Peatland,0.6,0.0,1c7ae3f8-25c3-4d54-b213-fbab34a3167c,6,6494.293060,328.608935,"POLYGON ((-6.1898 55.26712, -6.18979 55.26712,..."
6,Peatland,1.0,0.0,8130881c-028f-4183-8a12-e309c6d5ea13,7,10221.931641,436.762403,"POLYGON ((-6.19163 55.26585, -6.19176 55.26605..."
7,Peatland,0.3,0.0,6b8a0233-0d00-4a01-9751-2a2e9495174d,8,2655.300293,228.957951,"POLYGON ((-6.18934 55.26544, -6.1893 55.26539,..."
8,Peatland,1.5,0.0,aa610ef8-19c4-49f6-9eae-952b707e3bfc,9,15056.792450,847.657101,"POLYGON ((-6.19425 55.2632, -6.19368 55.26337,..."
9,Peatland,0.8,0.0,c025dee3-2011-4ca6-9245-66ee6b420ebe,10,7692.797287,426.930255,"POLYGON ((-6.19582 55.26923, -6.19636 55.26929..."


In [9]:
"""
Access and display the row of the peat GeoDataFrame with index label 10
    """
peat.loc[10]

Hab_Type                                               Peatland
Area_Hecta                                                  1.2
Area_Sq_Km                                                  0.0
GlobalID                   aa6beda0-70d8-4e75-b4fe-1ae486e007d0
OBJECTID                                                     11
Shape__Are                                         12067.550598
Shape__Len                                           636.433157
geometry      POLYGON ((-6.194743698171051 55.27015799242392...
Name: 10, dtype: object

In [10]:
peat['Area_Hecta'].sum()

np.float64(165043.4)

In [11]:
peat['Area_Sq_Km'].sum()

np.float64(1616.6)

In [12]:
peat['Shape__Are'].sum()

np.float64(1650399430.087551)

In [13]:
"""
Preview the first 10 rows of the aonb GeoDataFrame to inspect structure and key attributes
    """
aonb.head(10)

,Id,NAME,LEGISLATIN,DIST_COUNC,COUNTY,CALC_AREA,ID_REF,D_DEC_DATE,Area_Hecta,COMMENTS,GIS_AREA,Hyperlink,Shape_STAr,Shape_STLe,geometry
0,0,Mourne,NCALO,"Down, Newry & Mourne",Down,57965.79,AONB2,09/12/1986,57965.787376,None,57965.787376,https://www.daera-ni.gov.uk/articles/mourne-aonb,5.796579e+08,166316.826932,"MULTIPOLYGON (((-5.87484 54.15453, -5.87485 54..."
1,0,Strangford and Lecale,NCALO,"Ards, Down",Down,52503.69,AONB8,08/06/2010,52503.686735,Designation relates to the bounded area above ...,52503.686735,https://www.daera-ni.gov.uk/articles/strangfor...,5.250369e+08,208465.198119,"POLYGON ((-5.65019 54.58959, -5.64983 54.58947..."
2,0,Ring of Gullion,NCALO,Newry & Mourne,Armagh,15328.56,AONB5,12/12/1991,15328.561025,None,15328.561025,https://www.daera-ni.gov.uk/articles/ring-gull...,1.532856e+08,74096.116716,"POLYGON ((-6.43064 54.05877, -6.43088 54.05883..."
3,0,Causeway Coast,NCALO,"Moyle, Coleraine",Antrim,4212.96,AONB4,22/03/1989,4212.959009,None,4212.959009,https://www.daera-ni.gov.uk/articles/causeway-...,4.212959e+07,122055.448663,"MULTIPOLYGON (((-6.64878 55.20413, -6.64874 55..."
4,0,Antrim Coast and Glens,NCALO,"Ballymena, Ballymoney, Larne, Moyle",Antrim,72488.65,AONB3,07/07/1988,72488.654422,None,72488.654422,https://www.daera-ni.gov.uk/articles/antrim-co...,7.248865e+08,236127.535744,"MULTIPOLYGON (((-6.2496 55.212, -6.24949 55.21..."
5,0,Lagan Valley,Amenity Lands Act,"Belfast, Castlereagh, Lisburn","Antrim, Down",3880.73,AONB1,1965,3880.726075,None,3880.726075,https://www.daera-ni.gov.uk/articles/lagan-val...,3.880726e+07,37175.529401,"POLYGON ((-5.97582 54.54987, -5.97563 54.54947..."
6,0,Binevenagh,NCALO,"Coleraine, Limavady",Derry,16594.13,AONB6,19/06/2006,16594.126747,None,16594.126747,https://www.daera-ni.gov.uk/articles/binevenag...,1.659413e+08,98494.984596,"POLYGON ((-6.94244 55.10145, -6.94266 55.10169..."
7,0,Sperrin,NCALO,"Coleraine, Cookstown, Derry, Limavady, Maghera...","Derry, Tyrone",118205.57,AONB7,11/08/2008,118205.573252,None,118205.573252,https://www.daera-ni.gov.uk/articles/sperrin-aonb,1.182056e+09,215116.716972,"POLYGON ((-7.35987 54.67765, -7.35996 54.67788..."


In [14]:
"""
Preview the first 10 rows of the assi GeoDataFrame to inspect structure and key attributes
    """
assi.head(10)

,OBJECTID,REFERENCE,NAME,COUNTY,MAP_SCALE,SPECIESPT1,SPECIESPT2,HABITAT,EARTH_SCI,PARTIES,DECLAREDAY,DECLARE_HA,CONFIRMDAY,CONFIRM_HA,GIS_AREA,GIS_LENGTH,Hyperlink,Shape_STAr,Shape_STLe,geometry
0,4562,ASSI043,Rathlin Island - Ballygill North,Antrim,1:10000,Breeding bird assemblage,No ASSI feature present in this field category,Dry heath,No ASSI feature present in this field category,7.0,1991-09-16,78.00,1992-03-26,78.00,75.936678,4662.538215,https://www.daera-ni.gov.uk/publications/bally...,7.593668e+05,4662.538215,"POLYGON ((-6.23265 55.30948, -6.23269 55.30935..."
1,4563,ASSI044,Rathlin Island - Kinramer South,Antrim,1:10000,Pyramidal Bugle,No ASSI feature present in this field category,No ASSI feature present in this field category,No ASSI feature present in this field category,8.0,1991-09-16,25.00,1992-03-26,25.00,23.912271,2017.789353,https://www.daera-ni.gov.uk/publications/kinra...,2.391227e+05,2017.789353,"POLYGON ((-6.26462 55.29391, -6.26467 55.2939,..."
2,4564,ASSI033,Rathlin Island - Coast,Antrim,1:10000,"Common Gull, Fulmar, Guillemot, Herring Gull, ...",No ASSI feature present in this field category,"Subtidal, Coastal vegetated shingle, Maritime ...","Tertiary igneous, Tertiary igneous",29.0,1991-09-16,257.00,1992-03-26,257.00,236.063540,77916.507670,https://www.daera-ni.gov.uk/publications/rathl...,2.360635e+06,77916.507670,"MULTIPOLYGON (((-6.26307 55.3069, -6.26284 55...."
3,4565,ASSI050,Sheep Island,Antrim,1:10000,Great Cormorant,No ASSI feature present in this field category,No ASSI feature present in this field category,No ASSI feature present in this field category,2.0,1992-02-27,3.50,1992-07-08,3.50,3.495250,1282.752820,https://www.daera-ni.gov.uk/publications/sheep...,3.495250e+04,1282.752820,"POLYGON ((-6.35332 55.2483, -6.3533 55.24839, ..."
4,4566,ASSI116,Carrickarade,Antrim,1:10000,"Invertebrate assemblage, Breeding bird assemblage",No ASSI feature present in this field category,Maritime cliff and slopes,"Cretaceous stratigraphy, Tufa cascade, Tertiar...",4.0,1996-10-30,17.88,1997-03-13,17.88,17.799719,6939.272122,https://www.daera-ni.gov.uk/publications/carri...,1.779972e+05,6939.272122,"MULTIPOLYGON (((-6.32809 55.23586, -6.33015 55..."
5,4567,ASSI107,White Park Bay,Antrim,1:10000,"Higher plant assemblage, Invertebrate assemblage",No ASSI feature present in this field category,"Coastal sand dunes, Maritime cliff and slopes","Coastal processes, Cretaceous stratigraphy",19.0,1996-07-22,87.74,1997-02-26,87.74,88.263912,7923.478968,https://www.daera-ni.gov.uk/publications/white...,8.826391e+05,7923.478968,"MULTIPOLYGON (((-6.41478 55.23429, -6.41477 55..."
6,4568,ASSI202,Giant's Causeway and Dunseverick,Antrim,1:10000,"Higher plant assemblage, Invertebrate assembla...",No ASSI feature present in this field category,"Coastal saltmarsh, Coastal vegetated shingle, ...","Tertiary igneous, Mass movement",32.0,2000-02-25,226.33,2000-07-11,226.33,226.333083,46863.265109,https://www.daera-ni.gov.uk/publications/giant...,2.263331e+06,46863.265109,"MULTIPOLYGON (((-6.51692 55.23381, -6.51701 55..."
7,4569,ASSI175,Runkerry,Antrim,1:10000,No ASSI feature present in this field category,No ASSI feature present in this field category,No ASSI feature present in this field category,Coastal processes,9.0,1999-01-20,24.12,1999-08-19,24.12,24.120428,6255.886548,https://www.daera-ni.gov.uk/publications/runke...,2.412043e+05,6255.886548,"MULTIPOLYGON (((-6.53188 55.22984, -6.53156 55..."
8,4570,ASSI101,Portballintrae,Antrim,1:10000,No ASSI feature present in this field category,No ASSI feature present in this field category,No ASSI feature present in this field category,Pleistocene,3.0,1995-07-24,1.38,1995-12-20,1.38,1.129700,848.517693,https://www.daera-ni.gov.uk/publications/portb...,1.129700e+04,848.517693,"POLYGON ((-6.54855 55.2161, -6.54858 55.21581,..."
9,4571,ASSI174,White Rocks,Antrim,1:10000,No ASSI feature present in this field category,No ASSI feature present in this field category,No ASSI feature present in th

In [15]:
"""
Preview the first 10 rows of the trails GeoDataFrame to inspect structure and key attributes
    """
trails.head(10)

,ID,Name,SourceID,Source,GUID,Category,Type,Descriptio,On_Road,Length,Verified,ShowOnMap,County,UpdateDate,Trail_Usea,Infrastruc,geometry
0,0,Annalong Coastal Path,166,Outscape,{7a03aac4-145f-436f-9320-45b533fc5930},Single-Use Trail,Walking Trail,Short Walks (up to 5 miles),False,1.055094,None,Yes,Down,None,Walking,False,"LINESTRING Z (-5.89724 54.10924 0, -5.89702 54..."
1,0,Antrim Hills Way,104,Outscape,{c89d5023-1fe8-4def-b9d6-9efdc833540a},Single-Use Trail,Walking Trail,Long Walks (over 20 miles),False,23.081263,None,Yes,Antrim,None,Walking,False,"LINESTRING Z (-5.9557 54.96901 0, -5.9558 54.9..."
2,0,Ardress House - Ladies Mile,149,Outscape,{0a2fe306-5136-48e1-9a09-067c5dffe2d8},Single-Use Trail,Walking Trail,Short Walks (up to 5 miles),False,0.949078,None,Yes,Armagh,None,Walking,False,"LINESTRING Z (-6.59171 54.44418 0, -6.59168 54..."
3,0,Argory Blackwater River Walk,724,Outscape,{4b801c6c-fab1-4b9e-a475-e2c79be826bc},Single-Use Trail,Walking Trail,Short Walks (up to 5 miles),False,1.546865,None,Yes,Armagh,None,Walking,False,"LINESTRING Z (-6.65542 54.46494 0, -6.65556 54..."
4,0,Argory Lime Tree Walk,319,Outscape,{3c967a01-0486-4f4c-9924-2d9a888e087a},Single-Use Trail,Walking Trail,Short Walks (up to 5 miles),False,0.955559,None,Yes,Armagh,None,Walking,False,"LINESTRING Z (-6.65542 54.46494 0, -6.65556 54..."
5,0,Avish to Eagles Hill,566,Outscape,{964c4e8e-8cb6-4363-ba42-34a6e6e9cfff},Single-Use Trail,Walking Trail,Short Walks (up to 5 miles),False,3.089288,None,Yes,Londonderry,None,Walking,False,"LINESTRING Z (-6.8777 55.15018 0, -6.87753 55...."
6,0,Ballintoy to Bushmills,479,Outscape,{2344ddbd-bcd0-4a50-8041-b0ba32384d92},Single-Use Trail,Walking Trail,Medium Walks (5 to 20 miles),False,12.517231,None,Yes,Antrim,None,Walking,False,"MULTILINESTRING Z ((-6.41203 55.2327 0, -6.412..."
7,0,Ballykelly Bank,167,Outscape,{93a971ad-9379-430f-99c7-a7836f046197},Single-Use Trail,Walking Trail,Short Walks (up to 5 miles),False,2.144171,None,Yes,Londonderry,None,Walking,False,"LINESTRING Z (-7.04102 55.05566 0, -7.0416 55...."
8,0,Ballymacormick Point,589,Outscape,{817e0293-e543-4988-b4d9-4b906423b33a},Single-Use Trail,Walking Trail,Short Walks (up to 5 miles),False,2.084712,None,Yes,Down,None,Walking,False,"LINESTRING Z (-5.63656 54.66785 0, -5.63669 54..."
9,0,Ballymacran Bank,188,Outscape,{90483fc2-7369-41aa-91b8-18f44aa3d75e},Single-Use Trail,Walking Trail,Short Walks (up to 5 miles),False,1.437489,None,Yes,Londonderry,None,Walking,False,"LINESTRING Z (-7.02213 55.09582 0, -7.02211 55..."


In [16]:
"""
Preview the first 10 rows of the scheme_data GeoDataFrame to inspect structure and key attributes
    """
scheme_data.head(10)

,Shape__Are,Shape__Len,Id,SCHEME_LEV,BUSINESS_C,FIELD_COUN,FIELD_AREA,FEATURE_HA,geometry
0,1.957984e+09,1.142588e+07,31385,Higher,1843,22234,892.38811,0855d530967f190aaed599507679d1d4,"MULTIPOLYGON (((-6.17019 54.94529, -6.17078 54..."
1,1.163740e+09,1.625396e+07,31386,Wider,4388,22710,420.50471,1194c1ef36beb99a7a61b270b96532e8,"MULTIPOLYGON (((-6.35567 54.3367, -6.35294 54...."


In [17]:
def classify_peatland(peat, assi, aonb):
    """
    Intersects peat with ASSI and AONB to create protected layer.
    Remaining peat becomes unprotected layer.
    """
    peat_assi = gpd.overlay(peat, assi, how='intersection')    # peat inside ASSI
    peat_aonb = gpd.overlay(peat, aonb, how='intersection')    # peat inside AONB

    protected_peat = gpd.GeoDataFrame(pd.concat([peat_assi, peat_aonb], ignore_index=True), crs=peat.crs).copy()  # merge
    protected_peat["Area_ha"] = protected_peat.to_crs(epsg=3035).geometry.area / 10000  # calculate hectares (in EPSG:3035)

    unioned = protected_peat.geometry.union_all()  # create single protected area union
    unprotected_peat = peat[~peat.geometry.intersects(unioned)].copy()  # everything else outside protected peat
    unprotected_peat["Area_ha"] = unprotected_peat.to_crs(epsg=3035).geometry.area / 10000  # area in hectares

    return protected_peat, unprotected_peat


In [18]:
def create_interactive_map(protected_peat, unprotected_peat, trails, scheme_data):
    """
    Creates an interactive Folium map with multiple toggleable layers for:
    - Protected and unprotected peatlands
    - Off-road trails
    - Farm scheme data
    - Scheme data (e.g., farm location)
    Includes popups and tooltips for each layer
    """
    center = protected_peat.geometry.union_all().centroid  # get center of map based on peatlands

    m = folium.Map(location=[center.y, center.x], zoom_start=8)  # base map setup

    
    #        - Protected Peat-
    
    fg_protected = folium.FeatureGroup(name="Protected Peatlands", show=True)  # create layer group

    for _, row in protected_peat.iterrows():
        name = row.get("ASSI_NAME") or row.get("AONB_NAME") or row.get("NAME") or "Protected Peatland"  # get name
        area = round(row["Area_ha"], 2)  # get area
        folium.GeoJson(
            row.geometry,
            style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 1, 'fillOpacity': 0.5},  # style
            tooltip=name,  # on-hover name
            popup=folium.Popup(f"<b>{name}</b><br>Area: {area} ha", max_width=300)  # click popup
        ).add_to(fg_protected)

    fg_protected.add_to(m)  # add to map
    

    #        - Unprotected Peat -
    
    fg_unprotected = folium.FeatureGroup(name="Unprotected Peatlands", show=True)  # create toggle group

    for _, row in unprotected_peat.iterrows():
        name = row.get("ASSI_NAME") or row.get("AONB_NAME") or row.get("NAME") or "Protected Peatland"
        area = round(row["Area_ha"], 2)
        folium.GeoJson(
            row.geometry,
            style_function=lambda x: {'fillColor': 'darkviolet', 'color': 'darkviolet', 'weight': 1, 'fillOpacity': 0.4},
            tooltip="Unprotected Peatland",
            popup=folium.Popup(f"<b>Unprotected Peatland</b><br>Area: {area} ha", max_width=300)
        ).add_to(fg_unprotected)

    fg_unprotected.add_to(m)
    

    #         - Trails -
    
    fg_trails = folium.FeatureGroup(name="Off-Road Trails", show=True)

    folium.GeoJson(
        trails,
        style_function=lambda x: {'color': 'blue', 'weight': 1.2},
        tooltip=folium.GeoJsonTooltip(
        fields=['Name', 'Length'],
            aliases=['Trail Name:', 'Length (km):'],
            localize=True
        ),
        popup=folium.GeoJsonPopup(
            fields=['Name', 'Length'],
            aliases=['<b>Trail Name:</b>', '<b>Length (km):</b>'],
            localize=True,
            max_width=300
        )
    ).add_to(fg_trails)
    fg_trails.add_to(m)

    
    #        - Farms (Scheme Data) -
    
    
    fg_farms = folium.FeatureGroup(name="Farms (Scheme Data)", show=True)

    for _, row in scheme_data.iterrows():
        if row.geometry.is_empty:
            continue
        label = row.get("FARM_ID") or row.get("FARMREF") or row.get("Farm_Name") or "Farm"
        folium.GeoJson(
            row.geometry,
            style_function=lambda x: {'fillColor': 'green', 'color': 'green', 'weight': 1, 'fillOpacity': 0.2},
            tooltip=label,
            popup=folium.Popup(f"<b>Farm ID:</b> {label}", max_width=300)
        ).add_to(fg_farms)

    fg_farms.add_to(m)
    

    #         - Add Layer Control -
    
    folium.LayerControl().add_to(m)
    
    return m


In [19]:
"""
    Load Peatland Protection Map and save it as an html file
    """

# Load all datasets
peat, assi, aonb, trails, scheme_data = load_and_prepare_data()

# Classify protected status
protected_peat, unprotected_peat = classify_peatland(peat, assi, aonb)

# Build the map with layers
m = create_interactive_map(protected_peat, unprotected_peat, trails, scheme_data)

# Save map as HTML file
m.save("peatland_protection_map_interactive.html")
print("Map saved as: peatland_protection_map_interactive.html")


Map saved as: peatland_protection_map_interactive.html
